# 외국인 관광객 통계
#### 국적별로 외국인 관광객의 숫자에는 어떤 패턴이 있는지, 외국인 관광객이 증가한(또는 감소한) 원인이 되는 이벤트는 무엇인지 파악

# 외국인 출입국 통계 데이터 구하기 
#### 한국관광공사에서는 방한 와래관광객과 국민 해외관광객의 통계자료를 2003년 1월부터 매월 발표한다.
#### 이 자료는 성별, 목적별, 연령별, 교통수단별, 국적별로 집계되어 있음.
#### "월별 와래객 입국- 목적별/국적별" 통계 데이터 분석 및 시각화 
#### 2010년 1월부터 2019년 8월까지의 데이터를 엑셀로 저장해 두었음. 이를 활용해 분석

![ex_screenshot](./img/그림3-2.png)

#### 한국관광데이터랩
https://datalab.visitkorea.or.kr/datalab/portal/main/getMainForm.do

# 데이터 불러오기 및 전처리
#### 2010년 1월부터 2019년 8월가지 총 116개의 엑셀 파일

#### 목적별/국적별 외래객 데이터의 형태 
![ex_screenshot](img/그림3-4.png)
#### 중간생략
![ex_screenshot](img/그림3-4_1.png)

### 불러올 데이터의 형태 파악
#### 1. 첫 번째 row에는 제목에 해당하는 어느 시점의 어떤 종류의 데이터인지 서술(데이터 불러올 때 제외)
#### 2. 두 번째 row에는 총 10개의 컬럼이 순서대로 나열(A부터 G까지 활용)
#### 3. 3 - 69 번째 row에는 대륙, 국가에 따른 관광객 수 데이터가 구성되어 있음(실제 데이터 분석에 사용할 항목들임)
#### 4. 70 - 73 번째 row에는 계, 전년동기, 성장률, 구성비 등의 데이터 요약(데이터 불러올 때 제외)

In [1]:
# kto_201901.xlsx 엑셀 데이터 불러오기
import pandas as pd 
kto_201901 = pd.read_excel('./files/kto_201901.xlsx', 
                          header=1,         #컬럼이름은 두번빼 줄에 위치
                          usecols='A:G',    
                          skipfooter=4)     #마지막 4개의 row는 불러오지 않는다.
kto_201901.head()

,국적,관광,상용,공용,유학/연수,기타,계
0,아시아주,765082,10837,1423,14087,125521,916950
1,일본,198805,2233,127,785,4576,206526
2,대만,86393,74,22,180,1285,87954
3,홍콩,34653,59,2,90,1092,35896
4,마카오,2506,2,0,17,45,2570


### 데이터 전처리 하기
#### 데이터를 탐색하고, 분석에 적합하도록 원하는 형태로 가공한다.
#### 이런 과정을 데이터 전처리(Data Preprocessing), 데이터 가공(Data Manipulation), 데이터 클렌징(Data Cleansing), 또는 
#### 데이터 핸들링(Data Handling)이라고 한다.
#### 데이터 변수별로 값에 이상이없는지 확인, 결측값 처리, 이상치 처리, 변수 정규화, 파생변수 생성 등의 과정이 포함된다.

In [3]:
kto_201901.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   국적      67 non-null     object
 1   관광      67 non-null     int64 
 2   상용      67 non-null     int64 
 3   공용      67 non-null     int64 
 4   유학/연수   67 non-null     int64 
 5   기타      67 non-null     int64 
 6   계       67 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 3.8+ KB


### 데이터 전처리 과정 함수로 만들기 

In [27]:
# create_kto_data 함수 생성하기 
def create_kto_data(yy, mm):  
    #1. 불러올 Excel 파일 경로를 지정해주기
    file_path = './files/kto_{}{}.xlsx'.format(yy, mm)  
    
    # 2. Excel 파일 불러오기 
    df = pd.read_excel(file_path, header=1, skipfooter=4, usecols='A:G')
    
    # 3. "기준년월" 컬럼 추가하기 
    df['기준년월'] = '{}-{}'.format(yy, mm) 
    
    # 4. "국적" 컬럼에서 대륙 제거하고 국가만 남기기 
    ignore_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']    # 제거할 대륙명 선정하기 
    condition = (df['국적'].isin(ignore_list) == False)    # 대륙 미포함 조건 
    df_country = df[condition].reset_index(drop=True) 
    
    # 5. "대륙" 컬럼 추가하기 
    continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['대양주']*3 + ['아프리카']*2 + ['기타대륙'] + ['교포']    # 대륙 컬럼 생성을 위한 목록 만들어 놓기 
    df_country['대륙'] = continents   
                       
    # 6. 국가별 "관광객비율(%)" 컬럼 추가하기 
    df_country['관광객비율(%)'] = round(df_country.관광 / df_country.계 * 100, 1) 
                       
    # 7. "전체비율(%)" 컬럼 추가하기 
    tourist_sum = sum(df_country['관광'])
    df_country['전체비율(%)'] = round(df_country['관광'] / tourist_sum * 100, 1)
    
    # 8. 결과 출력하기 
    return(df_country)


In [28]:
# create_kto_data() 함수를 활용해서 2018년 12월 데이터 불러오기  
kto_test = create_kto_data(2018, 12)
kto_test.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
0,일본,252461,1698,161,608,3593,258521,2018-12,아시아,97.7,22.7
1,대만,85697,71,22,266,1252,87308,2018-12,아시아,98.2,7.7
2,홍콩,58355,41,3,208,939,59546,2018-12,아시아,98.0,5.2
3,마카오,6766,0,1,20,36,6823,2018-12,아시아,99.2,0.6
4,태국,47242,42,302,58,6382,54026,2018-12,아시아,87.4,4.2


### 반복문(for문)을 통해 다수의 엑셀 데이터 불러와서 합치기 

In [28]:
# 이중 반복문으로 기준년월 출력하기 
for yy in range(2010, 2020): 
    for mm in range(1, 13): 
        yymm = '{}{}'.format(yy, mm)
        print(yymm)

20101
20102
20103
20104
20105
20106
20107
20108
20109
201010
201011
201012
20111
20112
20113
20114
20115
20116
20117
20118
20119
201110
201111
201112
20121
20122
20123
20124
20125
20126
20127
20128
20129
201210
201211
201212
20131
20132
20133
20134
20135
20136
20137
20138
20139
201310
201311
201312
20141
20142
20143
20144
20145
20146
20147
20148
20149
201410
201411
201412
20151
20152
20153
20154
20155
20156
20157
20158
20159
201510
201511
201512
20161
20162
20163
20164
20165
20166
20167
20168
20169
201610
201611
201612
20171
20172
20173
20174
20175
20176
20177
20178
20179
201710
201711
201712
20181
20182
20183
20184
20185
20186
20187
20188
20189
201810
201811
201812
20191
20192
20193
20194
20195
20196
20197
20198
20199
201910
201911
201912


In [29]:
# 숫자 1 출력하기 
mm = 1 
print(mm) 

1


In [30]:
# zfill() 함수 이용하기 1(0을 포함한 두 자리로 출력하기)
print(str(mm).zfill(2))

01


In [31]:
# zfill() 함수 이용하기 1(0을 포함한 3자리, 4자리로 출력하기)
print(str(mm).zfill(3))
print(str(mm).zfill(4))

001
0001


In [32]:
# 6자리로 정렬하여 기준년월 출력하기 
for yy in range(2010, 2020):
    for mm in range(1, 13):
        mm_str = str(mm).zfill(2)
        yymm = '{}{}'.format(yy, mm_str)
        print(yymm)

201001
201002
201003
201004
201005
201006
201007
201008
201009
201010
201011
201012
201101
201102
201103
201104
201105
201106
201107
201108
201109
201110
201111
201112
201201
201202
201203
201204
201205
201206
201207
201208
201209
201210
201211
201212
201301
201302
201303
201304
201305
201306
201307
201308
201309
201310
201311
201312
201401
201402
201403
201404
201405
201406
201407
201408
201409
201410
201411
201412
201501
201502
201503
201504
201505
201506
201507
201508
201509
201510
201511
201512
201601
201602
201603
201604
201605
201606
201607
201608
201609
201610
201611
201612
201701
201702
201703
201704
201705
201706
201707
201708
201709
201710
201711
201712
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904
201905
201906
201907
201908
201909
201910
201911
201912


In [30]:
# 데이터를 담을 빈 데이터프레임 만들기 
df = pd.DataFrame()

In [31]:
# 패턴 분석결과를 이중 반복문을 통해 구현
for yy in range(2010, 2019):
    for mm in range(1, 13):
        temp = create_kto_data(str(yy), str(mm).zfill(2))
        df = df.append(temp, ignore_index=True)
        
for yy in range(2019, 2020):
    for mm in range(1, 9):
        temp = create_kto_data(str(yy), str(mm).zfill(2))
        df = df.append(temp, ignore_index=True)

In [32]:
# df 데이터 head() 함수로 확인하기
df.head()
#df.tail()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
0,일본,202825,1750,89,549,3971,209184,2010-01,아시아,97.0,50.6
1,대만,35788,41,17,37,516,36399,2010-01,아시아,98.3,8.9
2,홍콩,13874,55,0,21,595,14545,2010-01,아시아,95.4,3.5
3,마카오,554,0,0,0,0,554,2010-01,아시아,100.0,0.1
4,태국,13374,39,13,53,4335,17814,2010-01,아시아,75.1,3.3


In [33]:
# df 데이터 tail() 함수로 확인하기
df.tail()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
6955,대양주 기타,519,1,6,9,74,609,2019-08,대양주,85.2,0.0
6956,남아프리카공화국,634,6,1,11,1235,1887,2019-08,아프리카,33.6,0.1
6957,아프리카 기타,2081,593,165,841,1184,4864,2019-08,아프리카,42.8,0.2
6958,국적미상,36,1,0,8,12,57,2019-08,기타대륙,63.2,0.0
6959,교포,0,0,0,0,16560,16560,2019-08,교포,0.0,0.0


In [34]:
# 통합 데이터 생성하기 
df = pd.DataFrame()

# 패턴 분석결과를 이중 반복문을 통해 구현 
for yy in range(2010, 2020):
    for mm in range(1, 13):
        # try 이하 구문을 실행 
        try:
            # 앞서 만든 create_kto_data 함수로 Excel 데이터를 temp 에 불러오기 
            temp = create_kto_data(str(yy), str(mm).zfill(2))

            # temp 로 불러온 데이터를 df 밑으로 추가하기 
            df = df.append(temp, ignore_index=True)
        # 만약 try 구문에서 에러가 발생하면 except 이후를 실행 
        except:
            pass

In [35]:
# df 데이터 info() 함수로 확인하기 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6960 entries, 0 to 6959
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   국적        6960 non-null   object 
 1   관광        6960 non-null   int64  
 2   상용        6960 non-null   int64  
 3   공용        6960 non-null   int64  
 4   유학/연수     6960 non-null   int64  
 5   기타        6960 non-null   int64  
 6   계         6960 non-null   int64  
 7   기준년월      6960 non-null   object 
 8   대륙        6960 non-null   object 
 9   관광객비율(%)  6960 non-null   float64
 10  전체비율(%)   6960 non-null   float64
dtypes: float64(2), int64(6), object(3)
memory usage: 598.2+ KB


In [36]:
# df 데이터 엑셀로 저장하기
df.to_excel('./files/kto_total.xlsx', index = False)